In [46]:
import pandas as pd
import numpy as np

from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking, Embedding, TextVectorization
from tensorflow.data import Dataset

In [47]:
df = pd.read_csv('aspirin_like.csv')
df.drop_duplicates('Smiles',inplace = True)
df.shape

(269, 12)

In [48]:
smiles = df['Smiles'].to_list()
maxim, minim = len(smiles[0]), len(smiles[0])
for i in range(len(smiles)): #add newline to each SMILES string
    smiles[i] = smiles[i]+'\n'
smiles[1]

'C1=CC=C(C(=C1)C(=O)O)O[N+](=O)[O-]\n'

In [54]:
dataset = Dataset.from_tensor_slices(smiles)

TypeError: '_TensorSliceDataset' object is not subscriptable

In [55]:
preprocess = TextVectorization(standardize = None, split = 'character')
preprocess.adapt(smiles)

In [68]:
preprocess.get_vocabulary(include_special_tokens = False), preprocess.vocabulary_size()

(['C',
  '=',
  'O',
  ')',
  '(',
  '1',
  '\n',
  'N',
  'F',
  ']',
  '[',
  '-',
  'S',
  '+',
  'l',
  '2',
  'r',
  'B',
  'I',
  'H',
  '#'],
 23)